### Vizualizaciónes iniciales; 
pd: bueno aqui voy a tomar los datos y ver que ondita: Solo tome primer semestre 2025, hay muchos más periodos.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [2]:
# Cargar directamente con latin-1 (sino problemas de codificacion -\./-)
df_td = pd.read_csv('data/salud/sa2025/07OCTratoDirecto.csv', delimiter=';', encoding='latin-1')
df_cm = pd.read_csv('data/salud/sa2025/07OCConvenioMarco.csv', delimiter=';', encoding='latin-1')
df_lic = pd.read_csv('data/salud/sa2025/07OCLicitacion.csv', delimiter=';', encoding='latin-1')
df_ca = pd.read_csv('data/salud/sa2025/07OCCompraAgil.csv', delimiter=';', encoding='latin-1')

/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_35655/2135878628.py:3: DtypeWarning: Columns (10,11,12,17,28,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cm = pd.read_csv('data/salud/sa2025/07OCConvenioMarco.csv', delimiter=';', encoding='latin-1')
/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_35655/2135878628.py:4: DtypeWarning: Columns (11,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lic = pd.read_csv('data/salud/sa2025/07OCLicitacion.csv', delimiter=';', encoding='latin-1')
/var/folders/rn/8kg7t05x7l3fy1mwbgcdkw240000gp/T/ipykernel_35655/2135878628.py:5: DtypeWarning: Columns (7,8,13,35,38,42,43,49,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ca = pd.read_csv('data/salud/sa2025/07OCCompraAgil.csv', delimiter=';', encoding='latin-1')


In [3]:
# Función para limpiar y preparar los datos
def limpiar_datos_salud(df, nombre_dataset):
    """
    Limpia y prepara los datos de compras públicas del sector salud
    """
    df_clean = df.copy()
    
    # 1. Agregar identificador de dataset
    df_clean['Dataset'] = nombre_dataset
    
    # 2. Limpiar y convertir montos a numéricos
    if 'MontoTotalOC' in df_clean.columns:
        # Convertir a string y limpiar
        df_clean['MontoTotalOC'] = df_clean['MontoTotalOC'].astype(str)
        # Eliminar puntos de miles y reemplazar comas por puntos decimales
        df_clean['MontoTotalOC'] = df_clean['MontoTotalOC'].str.replace('.', '', regex=False)
        df_clean['MontoTotalOC'] = df_clean['MontoTotalOC'].str.replace(',', '.', regex=False)
        # Convertir a numérico
        df_clean['MontoTotalOC'] = pd.to_numeric(df_clean['MontoTotalOC'], errors='coerce')
    
    # 3. Limpiar otros campos de montos si existen
    campos_montos = ['MontoNetoOC', 'MontoNetoItem', 'MontoTotalItem', 'MontoNetoItemCLP']
    for campo in campos_montos:
        if campo in df_clean.columns:
            df_clean[campo] = df_clean[campo].astype(str)
            df_clean[campo] = df_clean[campo].str.replace('.', '', regex=False)
            df_clean[campo] = df_clean[campo].str.replace(',', '.', regex=False)
            df_clean[campo] = pd.to_numeric(df_clean[campo], errors='coerce')
    
    # 4. Limpiar fechas
    if 'FechaEnvioOC' in df_clean.columns:
        df_clean['FechaEnvioOC'] = pd.to_datetime(df_clean['FechaEnvioOC'], errors='coerce', dayfirst=True)
        df_clean['Año'] = df_clean['FechaEnvioOC'].dt.year
        df_clean['Mes'] = df_clean['FechaEnvioOC'].dt.month
        df_clean['Trimestre'] = df_clean['FechaEnvioOC'].dt.quarter
        df_clean['DiaSemana'] = df_clean['FechaEnvioOC'].dt.dayofweek
        df_clean['Semana'] = df_clean['FechaEnvioOC'].dt.isocalendar().week
    
    # 5. Limpiar campos de texto
    campos_texto = ['Proveedor', 'Institucion', 'DescripcionOC', 'NombreOC']
    for campo in campos_texto:
        if campo in df_clean.columns:
            df_clean[campo] = df_clean[campo].astype(str).str.strip()
            df_clean[campo] = df_clean[campo].replace('nan', '')
    
    # 6. Standardizar códigos
    if 'codigoOC' in df_clean.columns:
        df_clean['codigoOC'] = df_clean['codigoOC'].astype(str).str.strip()
    
    return df_clean

# Aplicar limpieza a todos los datasets
print("Limpiando datos del sector salud...")
df_td_clean = limpiar_datos_salud(df_td, 'Trato Directo')
df_cm_clean = limpiar_datos_salud(df_cm, 'Convenio Marco')
df_lic_clean = limpiar_datos_salud(df_lic, 'Licitación')
df_ca_clean = limpiar_datos_salud(df_ca, 'Compra Ágil')

# Verificar resultados de la limpieza
print("\n" + "="*50)
print("RESULTADOS DE LA LIMPIEZA")
print("="*50)

datasets = {
    'Trato Directo': df_td_clean,
    'Convenio Marco': df_cm_clean,
    'Licitación': df_lic_clean,
    'Compra Ágil': df_ca_clean
}

for nombre, df in datasets.items():
    print(f"\n{nombre}:")
    print(f"  • Registros: {len(df):,}")
    print(f"  • Columnas: {len(df.columns)}")
    
    if 'MontoTotalOC' in df.columns:
        monto_total = df['MontoTotalOC'].sum()
        monto_promedio = df['MontoTotalOC'].mean()
        print(f"  • Monto total: ${monto_total:,.0f} CLP")
        print(f"  • Monto promedio: ${monto_promedio:,.0f} CLP")
        print(f"  • Valores nulos en MontoTotalOC: {df['MontoTotalOC'].isnull().sum()}")
    
    if 'FechaEnvioOC' in df.columns:
        fechas_nulas = df['FechaEnvioOC'].isnull().sum()
        print(f"  • Fechas nulas: {fechas_nulas}")
        if fechas_nulas < len(df):
            fecha_min = df['FechaEnvioOC'].min()
            fecha_max = df['FechaEnvioOC'].max()
            print(f"  • Rango de fechas: {fecha_min} a {fecha_max}")

Limpiando datos del sector salud...

RESULTADOS DE LA LIMPIEZA

Trato Directo:
  • Registros: 11,788
  • Columnas: 50
  • Monto total: $185,054,146,866 CLP
  • Monto promedio: $15,698,519 CLP
  • Valores nulos en MontoTotalOC: 0
  • Fechas nulas: 0
  • Rango de fechas: 2025-01-02 00:00:00 a 2025-03-31 00:00:00

Convenio Marco:
  • Registros: 49,682
  • Columnas: 65
  • Monto total: $250,419,969,582 CLP
  • Monto promedio: $5,040,457 CLP
  • Valores nulos en MontoTotalOC: 0
  • Fechas nulas: 0
  • Rango de fechas: 2025-01-01 00:00:00 a 2025-03-31 00:00:00

Licitación:
  • Registros: 247,354
  • Columnas: 52
  • Monto total: $4,068,295,809,810 CLP
  • Monto promedio: $16,447,261 CLP
  • Valores nulos en MontoTotalOC: 0
  • Fechas nulas: 0
  • Rango de fechas: 2025-01-01 00:00:00 a 2025-03-31 00:00:00

Compra Ágil:
  • Registros: 605,008
  • Columnas: 69
  • Monto total: $1,598,198,297,372 CLP
  • Monto promedio: $2,641,615 CLP
  • Valores nulos en MontoTotalOC: 0
  • Fechas nulas: 0
  • 

In [4]:
# Explorar todas las columnas de cada dataset
print("="*70)
print("EXPLORACIÓN DE COLUMNAS POR DATASET")
print("="*70)

datasets = {
    'Trato Directo': df_td_clean,
    'Convenio Marco': df_cm_clean,
    'Licitación': df_lic_clean,
    'Compra Ágil': df_ca_clean
}

# Mostrar columnas de cada dataset
for nombre, df in datasets.items():
    print(f"\n{nombre.upper()}:")
    print(f"Total de columnas: {len(df.columns)}")
    print("-" * 50)
    
    # Mostrar columnas numeradas para facilitar referencia
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")

print("\n" + "="*70)
print("ANÁLISIS DE COLUMNAS COMUNES Y ÚNICAS")
print("="*70)

# Encontrar columnas comunes a todos los datasets
columnas_todas = [set(df.columns) for df in datasets.values()]
columnas_comunes = set.intersection(*columnas_todas)

print(f"\nColumnas comunes a TODOS los datasets ({len(columnas_comunes)}):")
for i, col in enumerate(sorted(columnas_comunes), 1):
    print(f"{i:2d}. {col}")

# Encontrar columnas únicas por dataset
print(f"\nColumnas ÚNICAS por dataset:")
for nombre, df in datasets.items():
    columnas_unicas = set(df.columns) - columnas_comunes
    if columnas_unicas:
        print(f"\n{nombre} ({len(columnas_unicas)} únicas):")
        for i, col in enumerate(sorted(columnas_unicas), 1):
            print(f"  {i:2d}. {col}")
    else:
        print(f"\n{nombre}: No tiene columnas únicas")

EXPLORACIÓN DE COLUMNAS POR DATASET

TRATO DIRECTO:
Total de columnas: 50
--------------------------------------------------
 1. codigoOC
 2. FechaEnvioOC
 3. NombreOC
 4. DescripcionOC
 5. EstadoOC
 6. ProcedenciaOC
 7. MonedaOC
 8. MontoNetoOC
 9. DescuentosOC
10. CargosOC
11. ImpuestosOC
12. MontoTotalOC
13. ImpuestosOC_CLP
14. MontoNetoOC_CLP
15. MetodoPago
16. TipoDespacho
17. Financiamiento
18. UnidadCompra
19. UnidadCompraRUT
20. RegionUnidadCompra
21. entCode
22. Institucion
23. Sector
24. Proveedor
25. ProveedorRUT
26. ActividadProveedor
27. TamanoProveedor
28. RegionProveedor
29. RubroN1
30. RubroN2
31. RubroN3
32. CodigoProductoONU
33. ONUProducto
34. NombreItem
35. DescripcionItem
36. CantidadItem
37. UnidadMedida
38. MonedaItem
39. MontoNetoItem
40. DescuentoItem
41. CargosItem
42. ImpuestoEspecificoItem
43. MontoTotalItem
44. MontoNetoItemCLP
45. Dataset
46. Año
47. Mes
48. Trimestre
49. DiaSemana
50. Semana

CONVENIO MARCO:
Total de columnas: 65
-------------------------

In [6]:


os.makedirs('../datos_limpios/salud', exist_ok=True)

# Definir las columnas para cada dataset
columnas_trato_directo = [
    # Identidad institucional
    'Institucion', 'UnidadCompra', 'RegionUnidadCompra',
    # Identidad proveedor
    'Proveedor', 'ProveedorRUT', 'TamanoProveedor', 'ActividadProveedor', 'RegionProveedor',
    # Monto
    'MontoTotalOC', 'MontoNetoOC_CLP', 'ImpuestosOC_CLP', 'DescuentosOC', 'CargosOC',
    # Producto o servicio
    'NombreItem', 'DescripcionItem', 'CantidadItem', 'UnidadMedida', 'MontoNetoItemCLP', 'CodigoProductoONU', 'ONUProducto',
    # Condiciones de pago
    'MetodoPago', 'TipoDespacho',
    # Tiempo
    'FechaEnvioOC', 'Año', 'Mes', 'Trimestre',
    # Clasificación del gasto
    'RubroN1', 'RubroN2',
    # Control de duplicados
    'codigoOC'
]

columnas_convenio_marco = [
    # Identidad institucional
    'Institucion', 'UnidadCompra', 'RegionUnidadCompra', 'Sector',
    # Identidad proveedor
    'Proveedor', 'ProveedorRUT', 'TamanoProveedor', 'ActividadProveedor', 'RegionProveedor',
    # Montos
    'MontoTotalOC', 'MontoNetoOC_CLP', 'ImpuestosOC_CLP', 'DescuentosOC', 'CargosOC',
    # Producto técnico
    'NombreItem', 'DescripcionItem', 'CantidadItem', 'MontoNetoItemCLP', 'CodigoProductoONU', 'ONUProducto', 'Producto', 'Modelo', 'Marca',
    # Condiciones
    'MetodoPago', 'CompraCoordinada', 'GranCompra', 'MontoAprobGC_TotalFinal',
    # Tiempo
    'FechaEnvioOC', 'Año', 'Mes', 'Trimestre',
    # Clasificación del gasto
    'RubroN1', 'RubroN2',
    # Evaluación
    'GlosaEvaluacion',
    # Control y trazabilidad
    'codigoOC', 'NroConvenioMarco'
]

columnas_licitacion = [
    # Identidad institucional
    'Institucion', 'UnidadCompra', 'RegionUnidadCompra', 'Sector',
    # Proveedor
    'Proveedor', 'ProveedorRUT', 'TamanoProveedor', 'ActividadProveedor', 'RegionProveedor',
    # Montos
    'MontoTotalOC', 'MontoNetoOC_CLP', 'ImpuestosOC_CLP', 'DescuentosOC', 'CargosOC',
    # Detalle producto/servicio
    'NombreItem', 'DescripcionItem', 'CantidadItem', 'MontoNetoItemCLP', 'CodigoProductoONU', 'ONUProducto',
    # Condiciones de compra
    'MetodoPago', 'CompraCoordinada', 'NroLicitacion',
    # Tiempo
    'FechaEnvioOC', 'Año', 'Mes', 'Trimestre',
    # Clasificación del gasto
    'RubroN1', 'RubroN2',
    # Trazabilidad
    'codigoOC'
]

columnas_compra_agil = [
    # Identidad institucional
    'Institucion', 'UnidadCompra', 'RegionUnidadCompra',
    # Proveedor y adjudicación
    'Proveedor', 'ProveedorRUT', 'TamanoProveedor', 'ProveedorCotizacion', 'ProveedorSeleccionado',
    # Montos OC y cotización
    'MontoTotalOC', 'MontoNetoOC_CLP', 'MontoNetoCotizacion', 'MontoTotalCotizacion', 'MontoDisponible', 'PrecioUnitarioCotizacion',
    # Detalle del producto
    'NombreItem', 'DescripcionItem', 'CantidadItem', 'MontoNetoItemCLP', 'ProductoCotizado', 'CantidadSolicitada', 'CodigoProductoONU',
    # Condiciones de compra
    'MetodoPago', 'CodigoCotizacion', 'DescripcionCotizacion',
    # Fechas y tiempo
    'FechaEnvioOC', 'FechaPublicacionCotizacion', 'FechaCierreCotizacion', 'Mes', 'Trimestre',
    # Clasificación del gasto
    'RubroN1', 'RubroN2',
    # Control / trazabilidad
    'codigoOC'
]

# Función para filtrar y guardar datasets
def filtrar_y_guardar(df, columnas_deseadas, nombre_archivo, nombre_modalidad):
    """
    Filtra el dataset con las columnas especificadas y lo guarda
    """
    # Identificar qué columnas están disponibles
    columnas_disponibles = [col for col in columnas_deseadas if col in df.columns]
    columnas_faltantes = [col for col in columnas_deseadas if col not in df.columns]
    
    print(f"\n{nombre_modalidad.upper()}:")
    print(f"  • Columnas solicitadas: {len(columnas_deseadas)}")
    print(f"  • Columnas disponibles: {len(columnas_disponibles)}")
    print(f"  • Columnas faltantes: {len(columnas_faltantes)}")
    
    if columnas_faltantes:
        print(f"  • Columnas NO encontradas: {columnas_faltantes}")
    
    # Filtrar el dataset
    df_filtrado = df[columnas_disponibles].copy()
    
    # Guardar el dataset filtrado
    ruta_archivo = f'datos_limpios/salud/{nombre_archivo}'
    df_filtrado.to_csv(ruta_archivo, index=False, sep=';', encoding='utf-8')
    
    print(f"  • Dataset guardado en: {ruta_archivo}")
    print(f"  • Registros: {len(df_filtrado):,}")
    print(f"  • Columnas finales: {len(df_filtrado.columns)}")
    
    return df_filtrado

print("="*70)
print("FILTRANDO Y GUARDANDO DATASETS")
print("="*70)

# Filtrar y guardar cada dataset
df_td_filtrado = filtrar_y_guardar(
    df_td_clean, 
    columnas_trato_directo, 
    'trato_directo_filtrado.csv', 
    'Trato Directo'
)

df_cm_filtrado = filtrar_y_guardar(
    df_cm_clean, 
    columnas_convenio_marco, 
    'convenio_marco_filtrado.csv', 
    'Convenio Marco'
)

df_lic_filtrado = filtrar_y_guardar(
    df_lic_clean, 
    columnas_licitacion, 
    'licitacion_filtrado.csv', 
    'Licitación'
)

df_ca_filtrado = filtrar_y_guardar(
    df_ca_clean, 
    columnas_compra_agil, 
    'compra_agil_filtrado.csv', 
    'Compra Ágil'
)

print("\n" + "="*70)
print("RESUMEN DE ARCHIVOS GUARDADOS")
print("="*70)



FILTRANDO Y GUARDANDO DATASETS

TRATO DIRECTO:
  • Columnas solicitadas: 29
  • Columnas disponibles: 29
  • Columnas faltantes: 0


OSError: Cannot save file into a non-existent directory: 'datos_limpios/salud'